In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('solutions_final')\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 11:17:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import col,row_number, sum ,format_number,desc,year,month,lag,when ,cast,avg
from pyspark.sql.window import Window
import pyspark.sql.functions as f

In [3]:
#reading the cleaned tables from the postgres database

jdbc_url = "jdbc:postgresql://localhost:5432/postgres"
connection_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}
table_name = "cleaned_data"
table_goods = "cleaned_goods"
table_services = "cleaned_services"
table_countries = "cleaned_countries"

df = spark.read \
    .jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

In [4]:
df_goods = spark.read \
    .jdbc(url=jdbc_url, table=table_goods, properties=connection_properties)

df_services = spark.read \
    .jdbc(url=jdbc_url, table=table_services, properties=connection_properties)

df_countries = spark.read \
    .jdbc(url=jdbc_url, table=table_countries, properties=connection_properties)

In [5]:
df.show()


+----------+-------+----+------------+------------+-----------+------+
|  time_ref|account|code|country_code|product_type|      value|status|
+----------+-------+----+------------+------------+-----------+------+
|2021-09-30|Imports|  24|          DK|       Goods|    17408.0|     F|
|2021-09-30|Imports|  24|          FJ|       Goods|        7.0|     F|
|2021-09-30|Imports|  24|          GB|       Goods|    10847.0|     F|
|2021-09-30|Imports|  24|          ID|       Goods|1.0549861E7|     F|
|2021-09-30|Imports|  24|          IN|       Goods|    17881.0|     F|
|2021-09-30|Imports|  24|          JO|       Goods|     2305.0|     F|
|2021-09-30|Imports|  24|          KR|       Goods|    91817.0|     F|
|2021-09-30|Imports|  24|          NL|       Goods|  2890316.0|     F|
|2021-09-30|Imports|  24|          TR|       Goods|    15015.0|     F|
|2021-09-30|Imports|  24|          US|       Goods|   102302.0|     F|
|2021-09-30|Imports|  25|          AE|       Goods|    34423.0|     F|
|2021-

In [6]:
df_countries.show()

+------------+--------------------+
|country_code|       country_label|
+------------+--------------------+
|          AD|             Andorra|
|          AE|United Arab Emirates|
|          AF|         Afghanistan|
|          AG| Antigua and Barbuda|
|          AI|            Anguilla|
|          AL|             Albania|
|          AM|             Armenia|
|          AO|              Angola|
|          AQ|          Antarctica|
|          AR|           Argentina|
|          AS|     Samoa, American|
|          AT|             Austria|
|          AU|           Australia|
|          AW|               Aruba|
|          AZ|          Azerbaijan|
|          BA|Bosnia and Herzeg...|
|          BB|            Barbados|
|          BD|          Bangladesh|
|          BE|             Belgium|
|          BF|        Burkina Faso|
+------------+--------------------+
only showing top 20 rows



In [7]:
df_goods.show()

+----+--------------------+
|code|      goods_category|
+----+--------------------+
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   1|       Animals; live|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   2|Meat and edible m...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
|   3|Fish and crustace...|
+----+--------------------+
only showing top 20 rows



In [8]:
df_services.show()

+-------+--------------------+
|   code|       service_label|
+-------+--------------------+
|    A12|            Services|
|  A1202|Maintenance and r...|
|  A1203|      Transportation|
| A12031|       Sea transport|
|A120312|             Freight|
|A120313|               Other|
| A12032|       Air transport|
|A120321|Air passenger tra...|
|A120322|Air freight trans...|
|A120323| Air other transport|
| A12034|Postal and courie...|
|  A1204|              Travel|
| A12041|     Business travel|
| A12042|     Personal travel|
|A120421|Education related...|
|A120422|Health related tr...|
|A120423|Other personal tr...|
|  A1205|Construction serv...|
|  A1206|Insurance and pen...|
| A12062|         Reinsurance|
+-------+--------------------+
only showing top 20 rows



### Question 1:
#### Trend analysis of the two countries which has the highest transactIons with new zealand. Increasing and decreasing trends of the sum of the transaction in each quarter month .

In [9]:
## top 2 countries with the highest number of transactions 
df_highest = df.groupBy('country_code').agg(sum(col('value')).alias('sum($)'))
df_highest= df_highest.orderBy(desc(col('sum($)')))
df_highest = df_highest.withColumn('sum($)', format_number(col('sum($)'),2)).limit(2)
print("the top 2 countries are : ",df_highest.collect()[0][0],":",df_highest.collect()[0][1],"and" , df_highest.collect()[1][0],':',df_highest.collect()[1][1] )

## only taking the transaction of the top 2 countires (filter first)
df_cn = df.filter(col('country_code')=='CN')
df_au = df.filter(col('country_code')=='AU')

columns = df_au.columns
for i in range(len(columns)):
    df_au = df_au.withColumnRenamed(columns[i], columns[i]+'_au')

## combining both countries transactions horizontally
df_joined = df_cn.join(df_au,df_cn['time_ref']==df_au['time_ref_au'],'inner')

#extracting month and year 
df_joined = df_joined.withColumn("year", year("time_ref")).withColumn("month", month("time_ref"))\
                     .withColumnRenamed('value','value($)')\
                     .withColumnRenamed('value_au','value($)_au')

df_joined.show()
## selecting only the required columns 
df_joined_req_col = df_joined.select('year','month','value($)','value($)_au')



df_joined_req_col = df_joined_req_col.groupBy('year','month')\
                        .agg(sum(col('value($)')).alias('sum($)'),sum(col('value($)_au')).alias('sum($)_au'))
df_joined_req_col.orderBy('year','month').show()

df_joined.printSchema()

#window
window_spec = Window.orderBy("year","month")

df_joined_req_col = df_joined_req_col.withColumn('lag($)',lag(col('sum($)'),1).over(window_spec))\
                                     .withColumn('lag_au($)',lag(col('sum($)_au'),1).over(window_spec))
df_joined_req_col = df_joined_req_col.na.drop()

#difference and trend analysis
df_final = df_joined_req_col.withColumn('difference($)',col('sum($)')-col('lag($)'))\
                            .withColumn('difference_au($)',col('sum($)_au')-col('lag_au($)'))


df_final = df_final.withColumn('change', when(col('difference($)')>0,"Increase").otherwise('Decrease'))\
                    .withColumn('change_au',when(col('difference_au($)')>0,'Increase').otherwise('Decrease'))

# df_final.show()difference_au


df_final_formatted = df_final.select('*')


for i in range(len(df_final.columns)-4):
    df_final_formatted = df_final_formatted.withColumn(df_final.columns[i+2],format_number(col(df_final.columns[i+2]),0))

df_final_formatted.show()




the top 2 countries are :  CN : 847,920,192,569.20 and AU : 761,016,231,517.13


+----------+-------+----+------------+------------+--------+------+-----------+----------+-------+---------------+---------------+-----------+---------+----+-----+
|  time_ref|account|code|country_code|product_type|value($)|status|time_ref_au|account_au|code_au|country_code_au|product_type_au|value($)_au|status_au|year|month|
+----------+-------+----+------------+------------+--------+------+-----------+----------+-------+---------------+---------------+-----------+---------+----+-----+
|2014-06-30|Exports|  71|          CN|       Goods| 14984.0|     F| 2014-06-30|   Imports|     48|             AU|          Goods|   231055.0|        F|2014|    6|
|2014-06-30|Exports|  71|          CN|       Goods| 14984.0|     F| 2014-06-30|   Imports|     48|             AU|          Goods|   117568.0|        F|2014|    6|
|2014-06-30|Exports|  71|          CN|       Goods| 14984.0|     F| 2014-06-30|   Imports|     48|             AU|          Goods|     4809.0|        F|2014|    6|
|2014-06-30|Expo

+----+-----+------------------+------------------+
|year|month|            sum($)|         sum($)_au|
+----+-----+------------------+------------------+
|2014|    6| 3.207832114424E13|3.0005845954049E13|
|2014|    9|3.0149345938005E13|3.3248377325064E13|
|2014|   12|3.7084530234212E13|3.5309257882759E13|
|2015|    3|3.5467973908136E13|3.1201448464812E13|
|2015|    6|3.3182651329464E13|3.1174360965062E13|
|2015|    9|3.7515060607995E13|3.3921927075906E13|
|2015|   12| 4.212451374876E13|3.4505529632608E13|
|2016|    3|   3.8452087044E13|3.1695938532378E13|
|2016|    6|3.7260607943702E13|3.1335067251072E13|
|2016|    9|3.6953105075259E13|3.3227853435588E13|
|2016|   12|4.3007005677438E13|3.4723597756813E13|
|2017|    3|4.0926173258734E13|3.2519835017932E13|
|2017|    6|4.3161959390616E13|3.2981432805654E13|
|2017|    9|4.1171007126056E13|3.6056523324192E13|
|2017|   12| 5.324741133542E13| 3.728352645622E13|
|2018|    3|4.5509936783506E13|  3.39461937882E13|
|2018|    6|4.7993021514571E13|

23/09/12 11:18:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:18:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:18:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:18:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:18:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:18:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 1

+----+-----+------------------+------------------+------------------+------------------+-------------------+------------------+--------+---------+
|year|month|            sum($)|         sum($)_au|            lag($)|         lag_au($)|      difference($)|  difference_au($)|  change|change_au|
+----+-----+------------------+------------------+------------------+------------------+-------------------+------------------+--------+---------+
|2014|    9|30,149,345,938,005|33,248,377,325,064|32,078,321,144,240|30,005,845,954,049| -1,928,975,206,235| 3,242,531,371,015|Decrease| Increase|
|2014|   12|37,084,530,234,212|35,309,257,882,759|30,149,345,938,005|33,248,377,325,064|  6,935,184,296,207| 2,060,880,557,695|Increase| Increase|
|2015|    3|35,467,973,908,136|31,201,448,464,812|37,084,530,234,212|35,309,257,882,759| -1,616,556,326,076|-4,107,809,417,947|Decrease| Decrease|
|2015|    6|33,182,651,329,464|31,174,360,965,062|35,467,973,908,136|31,201,448,464,812| -2,285,322,578,672|   -27,087

### Question 2 : 
#### TO calculate the most sold(Exported) good to each country in each year from NewZealand and  Find how deviated is it from the mean transaction  of that particular good. 


In [17]:

#finding just the sold goods
df_sold = df.filter(col('account')=='Exports')
df_sold = df_sold.filter(col('product_type')=='Goods')

#extracting year
df_each_year = df_sold.withColumn('year', year('time_ref'))


## finding out which country has imported max sum of  good in a year
df_result =df_each_year.groupBy('country_code','year').agg(f.max(col('value')).alias('value'))

#joining the main table with teh grouped one to find out which country imported how much in a particular year along with other details
df_joined = df_each_year.join(df_result , on = ['country_code','value','year'] , how = 'inner')
df_joined  = df_joined.dropDuplicates()
df_joined = df_joined.filter(col('code')!=00)
df_joined = df_joined.orderBy('country_code','year')
# df_joined.show()


## to find out the mean (average value) of each category of goods. 
df_goods_avg = df_each_year.groupBy('code').agg(avg(col('value')).alias('mean_value_goods'))
df_goods_avg = df_goods_avg.withColumn('mean_value_gpucho memeoods', f.round(col('mean_value_goods'),2))
df_goods_unique = df_goods.groupBy('goods_category').agg(f.max('code').alias('code'))
df_joined_avg = df_goods_avg.join(df_goods_unique , on = 'code', how='inner')
# df_joined_avg.show()



## joining the average for each good and previous table

df_joined_final =  df_joined.join(df_joined_avg, on = 'code', how = 'inner')

df_joined_final.orderBy('country_code','year').show()


##percent deviation from mean 

df_deviation = df_joined_final.withColumn('deviation from mean', f.round(col('value')-col('mean_value_goods'),2))

df_deviation.show()

+----+------------+-----------+----+----------+-------+------------+------+----------------+--------------------+
|code|country_code|      value|year|  time_ref|account|product_type|status|mean_value_goods|      goods_category|
+----+------------+-----------+----+----------+-------+------------+------+----------------+--------------------+
|  39|          AD|   543489.0|2018|2018-06-30|Exports|       Goods|     F|       388694.09|Plastics and arti...|
|  95|          AD|     6848.0|2020|2020-06-30|Exports|       Goods|     F|        119720.7|Toys, games and s...|
|  60|          AD|     2859.0|2021|2021-09-30|Exports|       Goods|     F|        49043.92|Fabrics; knitted ...|
|  87|          AD|    28366.0|2022|2022-12-30|Exports|       Goods|     F|       410770.22|Vehicles; other t...|
|  85|          AF|   969666.0|2019|2019-06-30|Exports|       Goods|     F|       333540.86|Electrical machin...|
|  72|          AF|    43511.0|2021|2021-09-30|Exports|       Goods|     F|      1282541

+----+------------+---------+----+----------+-------+------------+------+----------------+--------------------+-------------------+
|code|country_code|    value|year|  time_ref|account|product_type|status|mean_value_goods|      goods_category|deviation from mean|
+----+------------+---------+----+----------+-------+------------+------+----------------+--------------------+-------------------+
|  07|          PS|  96501.0|2014|2014-09-30|Exports|       Goods|     F|       701522.83|Vegetables and ce...|         -605021.83|
|  51|          IR|1330603.0|2021|2021-12-30|Exports|       Goods|     F|      1852744.33|Wool, fine or coa...|         -522141.33|
|  51|          LS|   4500.0|2018|2018-12-30|Exports|       Goods|     F|      1852744.33|Wool, fine or coa...|        -1848244.33|
|  87|          AD|  28366.0|2022|2022-12-30|Exports|       Goods|     F|       410770.22|Vehicles; other t...|         -382404.22|
|  87|          SM|   1548.0|2014|2014-12-30|Exports|       Goods|     F|   